# Experiment 1 Per-Layer Sensitivity

Quantize only one block at a time, keep others FP32

In [1]:
import torch
import yaml
from src.utils import *
from src.data_loader import get_data_loaders
from src.models.cnn_model_LayerWiseQuant import M5Modular, PTQM5Modular, PTQM5_LayerWiseQuant
from src.evaluate import test
fp_dict = torch.load("../models/cnn_fp32_model.pth")
ptq_dict = torch.load("../models/cnn_ptq_model.pth")
LWQ_dict_dicts = {
    1: torch.load("../models/cnn_ptq_LayerWiseQuant_q1_model.pth"),
    2: torch.load("../models/cnn_ptq_LayerWiseQuant_q2_model.pth"),
    3: torch.load("../models/cnn_ptq_LayerWiseQuant_q3_model.pth"),
    4: torch.load("../models/cnn_ptq_LayerWiseQuant_q4_model.pth"),
}

data_config = {
    "raw_dir": "../data/raw",
    "processed_dir": "./data/processed",
    "sample_rate": 8000,
    "batch_size": 256,
    "version": "v0.1"
}
train_loader, test_loader, _ = get_data_loaders(data_config)

/root/miniconda3/envs/audioml/lib/python3.13/site-packages/torch/_utils.py:431: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


In [2]:
# Load FP model
config_fp = '../configs/cnn_fp32.yaml'
with open(config_fp, 'r') as f:
    config = yaml.safe_load(f)
    
params_fp = config["model"]["base_cnn"]
model_fp = M5Modular(
        n_input=params_fp["n_input"],
        n_output=params_fp["n_output"],
        stride=params_fp["stride"],
        n_channel=params_fp["n_channel"],
        conv_kernel_sizes=params_fp["conv_kernel_sizes"]
        )
model_fp.load_state_dict(fp_dict)
model_fp.to('cpu')

# evaluate FP model
acc_fp = test(model_fp, test_loader)
print(f"FP32 model accuracy: {acc_fp:.4f}")

FP32 model accuracy: 83.0713


In [3]:
# Load fully quantized PTQ model
# Load PTQ model
config_PTQ = '../configs/cnn_ptq.yaml'
with open(config_PTQ, 'r') as f:
    config = yaml.safe_load(f)
    
params_PTQ = config["model"]["base_cnn"]
model_PTQ = PTQM5Modular(
            n_input=params_PTQ["n_input"],
            n_output=params_PTQ["n_output"],
            stride=params_PTQ["stride"],
            n_channel=params_PTQ["n_channel"],
            conv_kernel_sizes=params_PTQ["conv_kernel_sizes"]
        )
# Fuse and prepare for quantization
model_PTQ.eval()
model_PTQ.fuse_model()
model_PTQ.qconfig = torch.ao.quantization.get_default_qat_qconfig('x86')

model_PTQ.train()
torch.ao.quantization.prepare_qat(model_PTQ, inplace=True)

# Convert to quantized model
model_PTQ.eval()
model_PTQ = torch.ao.quantization.convert(model_PTQ, inplace=False)

# Load checkpoint
model_PTQ.load_state_dict(ptq_dict)
model_PTQ.to('cpu')

# evaluate PTQ model
acc_PTQ = test(model_PTQ, test_loader)
print(f"PTQ model accuracy: {acc_PTQ:.4f}")

/root/miniconda3/envs/audioml/lib/python3.13/site-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/root/miniconda3/envs/audioml/lib/python3.13/site-packages/torch/ao/quantization/utils.py:408: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


PTQ model accuracy: 75.8473


In [4]:

config_LWQ = '../configs/cnn_ptq_LayerWiseQuant.yaml'
with open(config_LWQ, 'r') as f:
    config = yaml.safe_load(f)

# for i in range(1, 2):
for i in config["model"]["quantization"]:
    model_LWQ = PTQM5_LayerWiseQuant(
        quantized_block_idx = i,
        n_input=config["model"]["base_cnn"]["n_input"],
        n_output=config["model"]["base_cnn"]["n_output"],
        stride=config["model"]["base_cnn"]["stride"],
        n_channel=config["model"]["base_cnn"]["n_channel"],
        conv_kernel_sizes=config["model"]["base_cnn"]["conv_kernel_sizes"],
    )

    # Fuse and prepare for quantization
    model_LWQ.eval()
    model_LWQ.fuse_model()
    model_LWQ.qconfig = torch.ao.quantization.get_default_qat_qconfig('x86')
    
    model_LWQ.train()
    torch.ao.quantization.prepare(model_LWQ, inplace=True)

    # Convert to quantized model
    model_LWQ.eval()
    model_LWQ = torch.ao.quantization.convert(model_LWQ, inplace=False)

    # # Load checkpoint
    model_LWQ.load_state_dict(LWQ_dict_dicts[i])
    model_LWQ.to('cpu')

    # evaluate single layer quantized model
    acc_LWQ = test(model_LWQ, test_loader)
    print(f"Layer-Wise Quantized Model (Layer {i} quantized) accuracy: {acc_LWQ:.4f}")


NotImplementedError: Could not run 'quantized::conv1d_relu' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv1d_relu' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMTIA, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at /pyto

In [ ]:
config_LWQ = '../configs/cnn_ptq_LayerWiseQuant.yaml'
with open(config_LWQ, 'r') as f:
    config = yaml.safe_load(f)

model_LWQ = PTQM5_LayerWiseQuant(
    quantized_block_idx = 1,
    n_input=config["model"]["base_cnn"]["n_input"],
    n_output=config["model"]["base_cnn"]["n_output"],
    stride=config["model"]["base_cnn"]["stride"],
    n_channel=config["model"]["base_cnn"]["n_channel"],
    conv_kernel_sizes=config["model"]["base_cnn"]["conv_kernel_sizes"],
)
model_LWQ.eval()
model_LWQ.fuse_model()
model_LWQ.qconfig = torch.ao.quantization.get_default_qat_qconfig('x86')

model_LWQ.train()
torch.ao.quantization.prepare_qat(model_LWQ, inplace=True)

# Convert to quantized model
model_LWQ.eval()
model_LWQ = torch.ao.quantization.convert(model_LWQ, inplace=False)

# Load checkpoint
model_LWQ.load_state_dict(LWQ_dict_dicts[i])
model_LWQ.to('cpu')

# evaluate single layer quantized model
acc_LWQ = test(model_LWQ, test_loader)
print(f"Layer-Wise Quantized Model (Layer {i} quantized) accuracy: {acc_LWQ:.4f}")


There are bugs in the previous commented code because of these reasons:

- Only quantized one layer (block1), but other layers are still in ```nn.Sequential``` with float ops.

- The fusion step (```fuse_model```) fused the layers as Conv + ReLU, so ```convert()``` tries to use ```Conv1dReLU``` as a fused quantized op.

- However, the input to that block is still float, since earlier layers are not quantized — and ```quantized::conv1d_relu``` expects quantized input tensors.

- *PyTorch’s eager mode quantization does not support partial layer quantization well* when fused operators like ```Conv1dReLU``` are involved.